## Testing hypothesis 1
---

**_Hypothesis_**: Reviews with longer text have higher helpfulness ratings.

   - **Metric**: Correlation coefficient (e.g., Pearson's correlation) between review length and helpfulness ratings. Plot the helpfulness rate as a function of the review length.

- **Missing Values**:

  - `review/text`: set missing values as empty string
  - `review/helpfulness`: remove the entire sample

- **Data Transformation**:

  - `review/text`: Count the number of words in each review removing punctuation and stopwords
  - `review/helpfulness`: $helpfulness = \frac{x}{y} \sqrt(y)$


In [ ]:
# Connect to MongoDB

import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['spark_db']
books = database['books_rating']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy_analyze import *

# Remove the samples which have no score or helpfulness data
pipeline_remove = {'$match':{
                        'review/text':{'$exists':True},
                        'N_helpful'	:{'$exists':True, '$ne':0},
                        'Tot_votes'	:{'$exists':True, '$ne':0}
                        }
                }

# Retain only the required fields
pipeline_project = {'$project':{
                            'review/text':1,
                            'review/helpfulness_rate':{'$multiply':[
                                                                {'$divide':['$N_helpful','$Tot_votes']},
                                                                {'$sqrt':'$Tot_votes'}
                                                                ]
                                                       },
                            'N_helpful':1,
                            'Tot_votes':1,
                            '_id':0,
                                }
                }

books_rating = books.aggregate([pipeline_remove,pipeline_project])

# Convert the cursor to a dataframe
books_rating = pd.DataFrame(list(books_rating))

books_rating.head()

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.tokenize import word_tokenize
import string
    
def get_tokenize_length(text):
    '''This function takes a string and returns the length of a list of tokens after removing stopwords and punctuations
    
    # Parameters
    Input: String
    Output: List of tokens
    
    '''
        
    words = word_tokenize(text)
    words = [word.lower() for word in words if word not in stopwords and word not in string.punctuation]
    
    return len(words)

# Tokenize the reviews
books_rating['review_length'] = books_rating['review/text'].apply(get_tokenize_length)

# Store in MongoDB
#books_1 = database['books_hypothesis_1']
#books_1.insert_many(books_rating.to_dict('records'))


In [ ]:
# Load the correct collection
books_1 = database['books_hypothesis_1']
books_rating = pd.DataFrame(books_1.find())
del books_rating['_id']
del books_rating['review/text']

# Filter out reviews with length less than 10
books_rating = books_rating[books_rating['review_length'] > 10]
books_rating = books_rating[books_rating['review/helpfulness_rate'] < 100]

# Filter out reviews with N_total_votes less than 20
# For the positive bias discovered in the hypothesis 3, we need to filter out the reviews with less than 20 votes to reduce the bias in the data
books_rating = books_rating[books_rating['Tot_votes'] > 20]

# Plot the distribution of review length with respect to helpfulness rate
books_rating.plot(kind='scatter',x='review_length',y='review/helpfulness_rate',figsize=(15,10),title='Review Length vs Helpfulness Rate')

# Analyze
scipy_analize(books_rating['review_length'],'review/length', books_rating['review/helpfulness_rate'], 'review/helpfulness_rate', [1,2])

## **First result**
#### In general we can see that there is a correlation between the length of the review and the helpfulness of the review, as supported by the correlation coefficient. However looking at the scatter plot, a different behavior is observed for reviews with length between 0 and 500 words. This must be further investigated.

---

#### We might think that the correlation is positive when the review has a reasonable length, but it is negative when the review is too long. We will divide the data into bins and analyze the correlation in each bin.

In [ ]:
# Create 5 bins of review length
groups = [0, 400, 750, 3000]
books_rating['length_bin'] = pd.cut(books_rating['review_length'], bins=groups, labels = [group for group in groups[1:]])
#books_rating.drop(books_rating.index[books_rating['length_bin'].isna().values], inplace=True)

import seaborn as sns

# Plot the distribution of review length with respect to helpfulness rate
plt.figure(figsize=(15,10))
sns.boxplot(x='length_bin', y='review/helpfulness_rate', data=books_rating, palette='rainbow')
plt.title('Review Length Range vs Helpfulness Rate')

import scipy

for el in groups[1:]:
    df = books_rating[books_rating['length_bin']==el]
    corr, pval = scipy.stats.kendalltau(df['review_length'], df['review/helpfulness_rate'])
    print(f'Group number: {el}\nCorrelation Coefficient: {corr}\nP-value: {pval}\n')
    plt.figure(figsize=(15,10))
    df.plot(kind='scatter',x='review_length',y='review/helpfulness_rate',figsize=(15,10),title=f'Review Length vs Helpfulness Rate in Group {el}')

## **Conclusions**
#### The correlation between the review length and the helpfulness changes inside the different groups. Specifically it is positive for reviews under 250 words, negative for reviews over 750 words, while it is quite neutral for the other groups. However, the correlation coefficient is low in all the groups, so we can conclude that there is no strong correlation between the length of the review and its helpfulness.